<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#hash_unique.unique-is-faster-than-numpy.unique-and-pandas.unique" data-toc-modified-id="hash_unique.unique-is-faster-than-numpy.unique-and-pandas.unique-1"><span class="toc-item-num">1&nbsp;&nbsp;</span><code>hash_unique.unique</code> is faster than <code>numpy.unique</code> and <code>pandas.unique</code></a></span></li><li><span><a href="#implementation" data-toc-modified-id="implementation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>implementation</a></span></li><li><span><a href="#Simple-test" data-toc-modified-id="Simple-test-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Simple test</a></span></li><li><span><a href="#Check-hash-quality" data-toc-modified-id="Check-hash-quality-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Check hash quality</a></span></li><li><span><a href="#performance-for-tiny-array" data-toc-modified-id="performance-for-tiny-array-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>performance for tiny array</a></span></li><li><span><a href="#performance-for-medium-array" data-toc-modified-id="performance-for-medium-array-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>performance for medium array</a></span></li><li><span><a href="#performance-for-large-array" data-toc-modified-id="performance-for-large-array-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>performance for large array</a></span></li></ul></div>

# `hash_unique.unique` is faster than `numpy.unique` and `pandas.unique`

for random array.

# implementation

In [1]:
%%file hash_unique.py

# don't remove the follows for your use
# Author: https://lhprojects.github.io/blog/
# don't remove above lines

import numba
import numpy as np


def unique(ar, weights = None, return_counts = False, return_hit_accuracy = False):
    '''
    ar: integer array
    return:
        uniques [, uniques_counts] [, hit_accuracy]
        hit_accuracy should be close to 1. unless
        the hash function has some quality problem
    '''
    if weights is not None:
        f,s,t = unique_impl64_w(ar, weights)
    else:
        f,s,t = unique_impl64(ar)
        
    if return_counts and return_hit_accuracy:
        return f, s, t
    elif return_counts:
        return f, s
    elif return_hit_accuracy:
        return f, t
    else:
        return f
    
def unique32(ar, weights = None, return_counts = False, return_hit_accuracy = False):
    '''
    ar: integer array
    NOTE: upper 32 bits ignored for hash function
    return:
        uniques [, uniques_counts] [, hit_accuracy]
        hit_accuracy should be close to 1. unless
        the hash function has some quality problem
    '''
    
    if weights is not None:
        f,s,t = unique_impl32_w(ar, weights)
    else:
        f,s,t = unique_impl32(ar)
    
    if return_counts and return_hit_accuracy:
        return f, s, t
    elif return_counts:
        return f, s
    elif return_hit_accuracy:
        return f, t
    else:
        return f

@numba.njit
def length(l):
    l = int(np.ceil(np.log2(l)))
    # 4*len(ar) > l > 2*len(ar)
    l = 2 << l
    return l

@numba.njit
def FNV_1_64(v):
    # https://en.wikipedia.org/wiki/Fowler%E2%80%93Noll%E2%80%93Vo_hash_function
    
    byte_mask = np.uint64(255)
    bs = np.uint64(v)
    x1 = (bs) & byte_mask
    x2 = (bs>>8) &byte_mask
    x3 = (bs>>16) &byte_mask
    x4 = (bs>>24) &byte_mask
    x5= (bs>>32) &byte_mask
    x6= (bs>>40) &byte_mask
    x7= (bs>>48) &byte_mask
    x8= (bs>>56) &byte_mask

    FNV_primer = np.uint64(1099511628211)
    FNV_bias = np.uint64(14695981039346656037)
    h = FNV_bias
    h = h*FNV_primer
    h = h^x1
    h = h*FNV_primer
    h = h^x2
    h = h*FNV_primer
    h = h^x3
    h = h*FNV_primer
    h = h^x4
    h = h*FNV_primer
    h = h^x5
    h = h*FNV_primer
    h = h^x6
    h = h*FNV_primer
    h = h^x7
    h = h*FNV_primer
    h = h^x8
    return h

@numba.njit
def FNV_1_32(v):
    
    byte_mask = np.uint64(255)
    bs = np.uint64(v)
    x1 = (bs) & byte_mask
    x2 = (bs>>8) &byte_mask
    x3 = (bs>>16) &byte_mask
    x4 = (bs>>24) &byte_mask

    FNV_primer = np.uint64(1099511628211)
    FNV_bias = np.uint64(14695981039346656037)
    h = FNV_bias
    h = h*FNV_primer
    h = h^x1
    h = h*FNV_primer
    h = h^x2
    h = h*FNV_primer
    h = h^x3
    h = h*FNV_primer
    h = h^x4
    return h

@numba.njit
def make_hash_table(ar):
    l = length(len(ar))    
    mask = l - 1      
    
    uniques = np.empty(l, dtype=ar.dtype)
    uniques_cnt = np.zeros(l, dtype=np.int_)
    return uniques, uniques_cnt, l, mask

@numba.njit
def make_hash_table_w(ar):
    l = length(len(ar))    
    mask = l - 1      
    
    uniques = np.empty(l, dtype=ar.dtype)
    uniques_cnt = np.zeros(l, dtype=np.int_)
    uniques_weight = np.zeros(l, dtype=np.float_)
    return uniques, uniques_cnt, uniques_weight, l, mask

@numba.njit
def set_item(uniques, uniques_cnt, mask, h, v, total, miss_hits, weight):
        
    index = (h & mask)

    # open address hash
    # great cache performance
    while True:
        if uniques_cnt[index] == 0:
            # insert new
            uniques_cnt[index] += weight
            uniques[index] = v
            total += 1
            break
        elif uniques[index] == v:
            uniques_cnt[index] += weight
            break
        else:
            miss_hits += 1
            index += 1
            index = index & mask
    return total, miss_hits
    
@numba.njit
def set_item_w(uniques, uniques_cnt, uniques_weights, mask, h, v, w, total, miss_hits):
        
    index = (h & mask)

    # open address hash
    # great cache performance
    while True:
        if uniques_cnt[index] == 0:
            # insert new
            uniques_cnt[index] += 1
            uniques_weights[index] += w
            uniques[index] = v
            total += 1
            break
        elif uniques[index] == v:
            uniques_cnt[index] += 1
            uniques_weights[index] += w
            break
        else:
            miss_hits += 1
            index += 1
            index = index & mask
    return total, miss_hits
    
@numba.njit
def concrete(ar, uniques, uniques_cnt, l, total):
    # flush the results in a concrete array            
    uniques_ = np.empty(total, dtype=ar.dtype)
    uniques_cnt_ = np.empty(total, dtype=np.int_)
    t = 0
    for i in range(l):
        if uniques_cnt[i] > 0:
            uniques_[t] = uniques[i]
            uniques_cnt_[t] = uniques_cnt[i]
            t += 1
    return uniques_, uniques_cnt_

@numba.njit
def concrete_w(ar, uniques, uniques_cnt, uniques_weight, l, total):
    # flush the results in a concrete array            
    uniques_ = np.empty(total, dtype=ar.dtype)
    uniques_cnt_ = np.empty(total, dtype=np.int_)
    uniques_weight_ = np.empty(total, dtype=np.float_)
    t = 0
    for i in range(l):
        if uniques_cnt[i] > 0:
            uniques_[t] = uniques[i]
            uniques_cnt_[t] = uniques_cnt[i]
            uniques_weight_[t] = uniques_weight[i]
            t += 1
    return uniques_, uniques_cnt_, uniques_weight_

def unique_factor(hash_function):
    
    @numba.njit
    def unique_impl(ar):

        uniques, uniques_cnt, l, mask = make_hash_table(ar)
        total = 0    
        miss_hits = 0    

        for v in ar:
            h = hash_function(v)
            total, miss_hits = set_item(uniques, uniques_cnt, mask, h, v, total, miss_hits, 1)

        uniques_, uniques_cnt_ = concrete(ar, uniques, uniques_cnt, l, total)

        if len(ar) == 0:
            hit_accuracy = np.nan
        else:
            hit_accuracy = len(ar)/((len(ar) + miss_hits)*1.0)
        return uniques_, uniques_cnt_, hit_accuracy
    
    return unique_impl


def unique_factor_w(hash_function):
    
    @numba.njit
    def unique_impl_w(ar, weights):

        uniques, uniques_cnt, uniques_weight, l, mask = make_hash_table_w(ar)
        total = 0    
        miss_hits = 0    

        for i, v in enumerate(ar):
            h = hash_function(v)
            w = weights[i]
            total, miss_hits = set_item_w(uniques, uniques_cnt, uniques_weight, mask, h, v, w, total, miss_hits)
            
        uniques_, uniques_cnt_, uniques_weight_ = concrete_w(ar, uniques, uniques_cnt, uniques_weight, l, total)

        if len(ar) == 0:
            hit_accuracy = np.nan
        else:
            hit_accuracy = len(ar)/((len(ar) + miss_hits)*1.0)
        return uniques_, uniques_weight_, hit_accuracy
    
    return unique_impl_w

unique_impl64 = unique_factor(FNV_1_64)
unique_impl32 = unique_factor(FNV_1_32)
unique_impl64_w = unique_factor_w(FNV_1_64)
unique_impl32_w = unique_factor_w(FNV_1_32)

Overwriting hash_unique.py


# Simple test

In [2]:
import hash_unique
import pandas
import numpy as np
import imp
imp.reload(hash_unique)

x = np.random.randint(10, size=10)

u, c = np.unique(x, return_counts=True)

hash_u, hash_c = hash_unique.unique(x, return_counts=True)
index = np.argsort(hash_u)
hash_u = hash_u[index]
hash_c = hash_c[index]


for u_, c_ in zip(u, c):
    print((u_, c_), end=" ")
print()

for u_, c_ in zip(hash_u, hash_c):
    print((u_, c_), end=" ")
print()

(0, 2) (3, 1) (4, 1) (5, 1) (6, 2) (7, 3) 
(0, 2) (3, 1) (4, 1) (5, 1) (6, 2) (7, 3) 


# Check hash quality

In [3]:
x = np.random.randint(5000, size=10000)
f, hit_accuracy = hash_unique.unique(x, return_hit_accuracy=True)

length = hash_unique.length(len(x))
y = np.array([hash_unique.FNV_1_64(v) for v in x], dtype=np.uint64) & np.uint64(length - 1)

print("%.3f"%hit_accuracy)
print(len(set(x)))
print(len(set(y)))

0.950
4329
4065


# performance for tiny array

In [4]:
x = np.random.randint(5, size=10)
%timeit np.unique(x, return_counts=True)
%timeit hash_unique.unique(x, return_counts=True)
%timeit hash_unique.unique32(x, return_counts=True)
%timeit pandas.unique(x)

13.1 µs ± 126 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
1.39 µs ± 5.25 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
The slowest run took 4.53 times longer than the fastest. This could mean that an intermediate result is being cached.
2.46 µs ± 1.78 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
22.6 µs ± 2.04 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [4]:
x = np.random.randint(5, size=10)
%timeit np.unique(x, return_counts=True)
%timeit hash_unique.unique(x, return_counts=True)
%timeit hash_unique.unique32(x, return_counts=True)
%timeit pandas.unique(x)

12.8 µs ± 26.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
1.42 µs ± 4.92 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
1.42 µs ± 3.34 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
24.1 µs ± 90 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# performance for medium array

In [72]:
x = np.random.randint(5000, size=10000)
%timeit np.unique(x, return_counts=True)
%timeit hash_unique.unique(x, return_counts=True)
%timeit hash_unique.unique32(x, return_counts=True)
weights = np.ones(len(x), dtype=np.float)
%timeit hash_unique.unique(x, weights = weights, return_counts=True)
%timeit hash_unique.unique32(x, weights = weights, return_counts=True)
%timeit pandas.unique(x)

408 µs ± 2.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
111 µs ± 384 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
82.4 µs ± 643 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
149 µs ± 13.2 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
102 µs ± 462 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
123 µs ± 1.36 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [5]:
x = np.arange(10000,0,-1)
%timeit np.unique(x, return_counts=True)
%timeit hash_unique.unique(x)
%timeit hash_unique.unique32(x, return_counts=True)
%timeit pandas.unique(x)

107 µs ± 670 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
99.8 µs ± 294 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
64.9 µs ± 399 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
122 µs ± 382 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [6]:
x = np.arange(0,10000)
%timeit np.unique(x, return_counts=True)
%timeit hash_unique.unique(x)
%timeit hash_unique.unique32(x)
%timeit pandas.unique(x)

83.6 µs ± 732 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
98.5 µs ± 765 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
65.4 µs ± 408 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
123 µs ± 703 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# performance for large array

In [7]:
x = np.random.randint(10000000//2, size=10000000)
%timeit np.unique(x, return_counts=True)
%timeit hash_unique.unique(x, return_counts=True)
%timeit hash_unique.unique32(x, return_counts=True)
weights = np.ones(len(x), dtype=np.float)
%timeit hash_unique.unique(x, weights = weights, return_counts=True)
%timeit hash_unique.unique32(x, weights = weights, return_counts=True)
%timeit pandas.unique(x)

705 ms ± 3.66 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
536 ms ± 11.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
407 ms ± 5.21 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
723 ms ± 29.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
576 ms ± 7.37 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
590 ms ± 26.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
